# Early Fusion: Concatenate Attention Features

This notebook performs **early fusion** by concatenating attention features from all models.

**Approach:**
1. Load features from all models (BERT, RoBERTa, DeBERTa, XLNet)
2. Concatenate features (fused features = concat of all model features)
3. Train classifiers on fused features
4. Evaluate and save results

**Output:** Fused features, predictions, probabilities, and results saved to Drive.


In [ ]:
# Setup (run previous notebooks first)
import sys
from pathlib import Path
import numpy as np

BASE_PATH = Path('/content/semeval-context-tree-modular')
DATA_PATH = Path('/content/drive/MyDrive/semeval_data')
sys.path.insert(0, str(BASE_PATH))

from src.storage.manager import StorageManager
from src.features.fusion import fuse_attention_features
from src.models.trainer import train_and_evaluate
from src.models.classifiers import get_classifier_dict

storage = StorageManager(
    base_path=str(BASE_PATH),
    data_path=str(DATA_PATH),
    github_path=str(BASE_PATH)
)

# Load splits for labels
train_ds = storage.load_split('train')
dev_ds = storage.load_split('dev')

print("✅ Setup complete!")


In [ ]:
# Model and task configurations
MODELS = ['bert', 'roberta', 'deberta', 'xlnet']
TASKS = ['clarity', 'evasion']

# Label mappings
CLARITY_LABELS = ['Clear Reply', 'Ambiguous', 'Clear Non-Reply']
EVASION_LABELS = ['Direct Answer', 'Partial Answer', 'Implicit Answer', 
                  'Uncertainty', 'Refusal', 'Clarification', 
                  'Question', 'Topic Shift', 'Other']

# Get classifiers
classifiers = get_classifier_dict(random_state=42)
print(f"✅ Classifiers: {list(classifiers.keys())}")


In [ ]:
# Early fusion: Concatenate features from all models
for task in TASKS:
    print(f"\n{'='*80}")
    print(f"TASK: {task.upper()} - EARLY FUSION")
    print(f"{'='*80}")
    
    # Get label list
    if task == 'clarity':
        label_list = CLARITY_LABELS
        label_key = 'clarity_label'
    else:  # evasion
        label_list = EVASION_LABELS
        label_key = 'evasion_label'
    
    # Load features from all models
    print("Loading features from all models...")
    model_features = {}
    model_feature_names = {}
    
    for model in MODELS:
        X_train = storage.load_features(model, task, 'train')
        X_dev = storage.load_features(model, task, 'dev')
        
        # Get feature names from metadata
        meta = storage.load_metadata(model, task, 'train')
        feature_names = meta['feature_names']
        
        model_features[model] = {
            'train': X_train,
            'dev': X_dev
        }
        model_feature_names[model] = feature_names
        
        print(f"  {model}: {X_train.shape[1]} features")
    
    # Fuse features (concatenate)
    print("\nFusing features (concatenation)...")
    X_train_fused, fused_feature_names = fuse_attention_features(
        {model: model_features[model]['train'] for model in MODELS},
        model_feature_names
    )
    X_dev_fused, _ = fuse_attention_features(
        {model: model_features[model]['dev'] for model in MODELS},
        model_feature_names
    )
    
    print(f"  Fused features: {X_train_fused.shape[1]} features")
    print(f"  Train: {X_train_fused.shape[0]} samples")
    print(f"  Dev: {X_dev_fused.shape[0]} samples")
    
    # Save fused features
    storage.save_fused_features(
        X_train_fused, MODELS, task, 'train',
        fused_feature_names, fusion_method='concat'
    )
    storage.save_fused_features(
        X_dev_fused, MODELS, task, 'dev',
        fused_feature_names, fusion_method='concat'
    )
    
    # Get labels
    y_train = np.array([train_ds[i][label_key] for i in range(len(train_ds))])
    y_dev = np.array([dev_ds[i][label_key] for i in range(len(dev_ds))])
    
    # Train and evaluate on fused features
    print("\nTraining classifiers on fused features...")
    results = train_and_evaluate(
        X_train_fused, y_train, X_dev_fused, y_dev,
        label_list=label_list,
        task_name=f"early_fusion_{task}",
        classifiers=classifiers,
        random_state=42,
        print_report=True,
        print_table=True,
        create_plots=True,
        save_plots_dir=str(DATA_PATH / 'plots' / 'early_fusion')
    )
    
    # Save predictions and probabilities
    for classifier_name, result in results.items():
        # Save predictions
        storage.save_predictions(
            result['dev_pred'],
            'fused', classifier_name, task, 'dev'
        )
        
        # Save probabilities
        if result['dev_proba'] is not None:
            storage.save_probabilities(
                result['dev_proba'],
                'fused', classifier_name, task, 'dev'
            )
    
    # Save results summary
    experiment_id = f"early_fusion_{task}"
    storage.save_results({
        'fusion_method': 'early_concat',
        'models': MODELS,
        'task': task,
        'results': {
            name: {
                'metrics': res['metrics'],
                'n_train': len(y_train),
                'n_dev': len(y_dev)
            }
            for name, res in results.items()
        }
    }, experiment_id)

print(f"\n{'='*80}")
print("✅ Early fusion complete!")
print(f"{'='*80}")
